# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path = 'data/'
df = pd.read_csv(data_path + 'titanic_train.csv')
#df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
df.groupby('Ticket').size()

Ticket
110152         3
110413         3
110465         2
110564         1
110813         1
              ..
W./C. 6608     4
W./C. 6609     1
W.E.P. 5734    1
W/C 14208      1
WE/P 5735      2
Length: 681, dtype: int64

In [3]:
temp = df['Ticket'].apply(lambda x:str(x).find('110152'))
df[temp!=-1]

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
257,1,"Cherry, Miss. Gladys",female,30.0,0,0,110152,86.5,B77,S
504,1,"Maioni, Miss. Roberta",female,16.0,0,0,110152,86.5,B79,S
759,1,"Rothes, the Countess. of (Lucy Noel Martha Dye...",female,33.0,0,0,110152,86.5,B77,S


In [4]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""
mean_df = df.groupby('Ticket')['Fare'].mean().reset_index()
mode_df = df.groupby(['Ticket'])['Fare'].apply(lambda x: x.mode()[0]).reset_index()
median_df = df.groupby(['Ticket'])['Fare'].median().reset_index()
max_df = df.groupby(['Ticket'])['Fare'].max().reset_index()
temp = pd.merge(mean_df, mode_df, how='left', on=['Ticket'])
temp = pd.merge(temp, median_df, how='left', on=['Ticket'])
temp = pd.merge(temp, max_df, how='left', on=['Ticket'])
temp.columns = ['Ticket', 'Fare_Ticket_Mean', 'Fare_Ticket_Mode', 'Fare_Ticket_Median', 'Fare_Ticket_Max']
temp

,Ticket,Fare_Ticket_Mean,Fare_Ticket_Mode,Fare_Ticket_Median,Fare_Ticket_Max
0,110152,86.500,86.500,86.500,86.500
1,110413,79.650,79.650,79.650,79.650
2,110465,52.000,52.000,52.000,52.000
3,110564,26.550,26.550,26.550,26.550
4,110813,75.250,75.250,75.250,75.250
...,...,...,...,...,...
676,W./C. 6608,34.375,34.375,34.375,34.375
677,W./C. 6609,7.550,7.550,7.550,7.550
678,W.E.P. 5734,61.175,61.175,61.175,61.175
679,W/C 14208,10.500,10.500,10.500,10.500


In [5]:
df_merge = pd.merge(df, temp, how='left', on = ['Ticket'])

In [6]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df_merge.dtypes, df_merge.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df_merge = df_merge[num_features]
df_merge = df_merge.fillna(-1)
MMEncoder = MinMaxScaler()
df_merge.head()

9 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Fare_Ticket_Mean', 'Fare_Ticket_Mode', 'Fare_Ticket_Median', 'Fare_Ticket_Max']



,Pclass,Age,SibSp,Parch,Fare,Fare_Ticket_Mean,Fare_Ticket_Mode,Fare_Ticket_Median,Fare_Ticket_Max
0,3,22.0,1,0,7.2500,7.2500,7.2500,7.2500,7.2500
1,1,38.0,1,0,71.2833,71.2833,71.2833,71.2833,71.2833
2,3,26.0,0,0,7.9250,7.9250,7.9250,7.9250,7.9250
3,1,35.0,1,0,53.1000,53.1000,53.1000,53.1000,53.1000
4,3,35.0,0,0,8.0500,8.0500,8.0500,8.0500,8.0500


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [7]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
df_o = df_merge.drop(['Fare_Ticket_Mean', 'Fare_Ticket_Mode', 'Fare_Ticket_Median', 'Fare_Ticket_Max'] , axis=1)

train_X = MMEncoder.fit_transform(df_o)

logistic = LogisticRegression()
cross_val_score(logistic, train_X, train_Y, cv=5).mean()

0.70041428661101

In [8]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
train_X = MMEncoder.fit_transform(df_merge)

logistic = LogisticRegression()
cross_val_score(logistic, train_X, train_Y, cv=5).mean()

0.7037976272675914